Scrape a manula document, put results in a postgres database

In [2]:
import requests
import json
from dotenv import dotenv_values
from bs4 import BeautifulSoup
from base64 import b64encode
import psycopg
from dataclasses import dataclass,asdict
from psycopg.rows import class_row
import re

rootdoc = dotenv_values()["MANULAROOT"] 
rootdoc = "" if rootdoc == None else rootdoc
password = dotenv_values()["POSTGRES_PASSWORD"]


In [13]:
@dataclass
class WebArticle:
    url:str
    title:str
    contents:str
    hasvectors:bool = False
    def dict(self):
        return {k: str(v) for k, v in asdict(self).items()}

In [14]:
def manularootlinks(url):
    resp = requests.get(url)
    content = BeautifulSoup(resp.content).findAll("div",{"class":"box-toc"})
    links = [url]
    for div in content:
        l= div.findAll('a')
        for link in l:
            if "href" in link.attrs:
                url = link.attrs['href']
                if not re.search('/pdf$',url): links.append(link.attrs['href'])
    return links
def scrapeManula(url,limit=None):
    links = manularootlinks(url)
    arts = []
    count = 0
    for link in links:
        count+=1
        if limit!= None and count > limit : break
        resp = requests.get(link)
        soup = BeautifulSoup(resp.content)
        content = soup.find("div", id="content")
        title =soup.find("title")
        title = link if title == None else title.get_text()
        if content == None: continue
        arts.append(WebArticle( url = link,title = title, contents = str(content.get_text("\n"))))
    return arts

In [19]:
def connect(dbname="ragtest",autocommit=False):
        return psycopg.connect(f'host=pgvector dbname={dbname} user=postgres password={password}',autocommit=autocommit)
def createTable():
        with connect() as conn:
            with conn.cursor() as cur:
                cur.execute("""
                CREATE TABLE IF NOT EXISTS public.webarticles (
                    url character varying(250) NOT NULL,
                    title    text NOT NULL,
                    contents text NOT NULL,
                    hasvectors boolean,        
                    CONSTRAINT webarticles_pkey PRIMARY KEY (url)
                )
                TABLESPACE pg_default;
                """)
def insertArticles(articles:list[WebArticle]):
      with connect() as conn:
            with conn.cursor() as cur:
                  for art in articles:
                        cur.execute("""INSERT INTO public.webarticles (url, title,contents,hasvectors) VALUES (%(url)s,%(title)s,%(contents)s,%(hasvectors)s);""",art.dict())
createTable()

In [ ]:

articles = scrapeManula(rootdoc,10)
insertArticles(articles)